# Set run directory

In [50]:
from pathlib import Path
runDir = Path('example.laplace3d_ajacobi_2018-04-26_11-03-36')
runDir.exists()

True

# Update records

In [51]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Open records
records = pd.DataFrame()
if Path('records.csv').exists():
    records = pd.read_csv('records.csv')

with (runDir / '0.json').open('r') as recF: 
    data = load(recF)
    del data['CV sample times [s]']
    del data['CV samples']
    
    records = records.append(data, ignore_index=True)
records.tail()

,Device,Domain partitions,Exact solution,Executable name,Executable path,Info,Initial guess,Iterant,Matrix,Nodes,Processes,Processes grid,Processes per node,Rank,Relative error,Relative residual,Threads per process,Timeout [min],Timestamp,Tolerance
0,julia,"[[0, 256], [0, 256], [0, 256]]",white-noise sine,example.laplace3d_ajacobi,~/asynchronator/build/example,SIAM,zero,ajacobi,laplace3d,1.0,1.0,"[1, 1, 1]",1.0,0.0,0.550488,0.002674,256.0,2.0,2018-04-26 11:03:24,0.0001
1,julia,"[[0, 256], [0, 256], [0, 256]]",white-noise sine,example.laplace3d_ajacobi,~/asynchronator/build/example,SIAM,zero,ajacobi,laplace3d,1.0,1.0,"[1, 1, 1]",1.0,0.0,0.550961,0.002141,256.0,2.0,2018-04-26 11:03:36,0.0001


# Save records

In [52]:
records.to_csv('records.csv', index=False)

# Read iterations record

In [53]:
# Read files
itersData = []
for runP in runDir.glob('*.json'):
    if runP.name == 'setup.json': 
        continue
    with runP.open('r') as recF: 
        data = load(recF)
        dataDf = pd.DataFrame(data['CV samples'], 
                              index=data['CV sample times [s]'],
                              columns =['Rank {}'.format(runP.name[:-5])])
        dataDf.index.name = 'Time [s]'
        itersData.append(dataDf)

itersDf = pd.concat(itersData, axis=1)
itersDf.head()

,Rank 0
Time [s],
0.000000,1.000000
0.006033,1.000000
0.086071,1.000000
0.175370,0.814507
0.259820,0.788891


# Save iterations record

In [54]:
itersDf.to_csv('{}.csv'.format(runDir.name))

# Clean run directory

In [55]:
for f in runDir.glob('*'):
    f.unlink()
runDir.rmdir()